# 🔮 DataLexir - Démonstration Complète

Ce notebook démontre l'utilisation complète de DataLexir pour l'exploration de données et la génération de pipelines.

## 📋 Sommaire

1. [Installation et Import](#installation)
2. [Préparation des Données](#données)
3. [Exploration avec Elexbook](#exploration)
4. [Génération du Pipeline](#pipeline)
5. [Application avec Elexdas (Pandas)](#elexdas)
6. [Application avec Elexpark (Spark)](#elexpark)
7. [Conclusion](#conclusion)

<a id="installation"></a>
## 1. 📦 Installation et Import

Commençons par installer et importer les bibliothèques nécessaires.

In [ ]:
# Installation (si nécessaire)
# !pip install datalexir

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import DataLexir
from datalexir.elexbook import Elexbook
# from datalexir.elexdas import Elexdas
# from datalexir.elexpark import Elexpark

print("✅ Imports réussis !")

<a id="données"></a>
## 2. 📊 Préparation des Données

Créons un dataset de démonstration simulant des données de ventes e-commerce.

In [ ]:
# Génération d'un dataset de démonstration
np.random.seed(42)
n_records = 100000  # 100k enregistrements pour simuler un gros dataset

# Génération des données
data = {
    'transaction_id': [f'T{i:06d}' for i in range(1, n_records + 1)],
    'customer_id': [f'C{np.random.randint(1, 10000):05d}' for _ in range(n_records)],
    'product_category': np.random.choice(['Electronics', 'Clothing', 'Books', 'Home', 'Sports', None], n_records, p=[0.25, 0.2, 0.15, 0.2, 0.15, 0.05]),
    'amount': np.random.lognormal(3, 1, n_records),
    'quantity': np.random.randint(1, 10, n_records),
    'date': [datetime(2024, 1, 1) + timedelta(days=np.random.randint(0, 365)) for _ in range(n_records)],
    'payment_method': np.random.choice(['Credit Card', 'PayPal', 'Bank Transfer', 'Cash'], n_records, p=[0.5, 0.3, 0.15, 0.05]),
    'discount_applied': np.random.choice([True, False], n_records, p=[0.3, 0.7]),
    'customer_rating': np.random.choice([1, 2, 3, 4, 5, None], n_records, p=[0.05, 0.1, 0.15, 0.35, 0.3, 0.05])
}

# Ajout de quelques valeurs manquantes
missing_indices = np.random.choice(n_records, size=int(n_records * 0.02), replace=False)
for idx in missing_indices:
    data['amount'][idx] = None

# Création du DataFrame
sales_df = pd.DataFrame(data)

print(f"📊 Dataset généré : {len(sales_df):,} lignes x {len(sales_df.columns)} colonnes")
print(f"💾 Taille en mémoire : {sales_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
sales_df.head()

In [ ]:
# Aperçu du dataset
print("📈 Informations sur le dataset :")
print(sales_df.info())
print("\n📊 Statistiques descriptives :")
sales_df.describe()

<a id="exploration"></a>
## 3. 🔍 Exploration avec Elexbook

Maintenant, utilisons Elexbook pour explorer ce dataset sur un échantillon plus petit.

In [ ]:
# Création d'un Elexbook avec 5% du dataset
book = Elexbook(sales_df, sample=0.05)

print(f"📖 Elexbook créé !")
print(f"📊 Dataset original : {len(book.df):,} lignes")
print(f"🎯 Échantillon : {len(book.sample):,} lignes ({len(book.sample)/len(book.df)*100:.1f}%)")
print(f"📝 Historique : {len(book.history)} opérations")

In [ ]:
# Exploration de base
print("🔍 Aperçu de l'échantillon :")
book.head()

In [ ]:
# Vérification des valeurs manquantes
print("❓ Valeurs manquantes dans l'échantillon :")
book.isnull().sum()

In [ ]:
# Nettoyage des données
print("🧹 Nettoyage des données...")

# Suppression des lignes avec des montants manquants
book.dropna(subset=['amount'])
print(f"✅ Suppression des montants manquants : {len(book.sample)} lignes restantes")

# Remplissage des catégories manquantes
book.fillna({'product_category': 'Unknown'})
print(f"✅ Remplissage des catégories manquantes")

# Remplissage des notes manquantes avec la médiane
median_rating = book.sample['customer_rating'].median()
book.fillna({'customer_rating': median_rating})
print(f"✅ Remplissage des notes manquantes avec la médiane ({median_rating})")

In [ ]:
# Conversions de types
print("🔄 Conversion des types...")

# Conversion des types
book.astype({
    'amount': 'float64',
    'customer_rating': 'int64',
    'discount_applied': 'bool'
})

print("✅ Types convertis")
print("\n📊 Types après conversion :")
print(book.sample.dtypes)

In [ ]:
# Analyse exploratoire
print("📈 Analyse par catégorie de produit :")
category_analysis = book.groupby('product_category')['amount'].agg(['count', 'mean', 'sum']).round(2)
print(category_analysis)

print("\n💳 Analyse par méthode de paiement :")
payment_analysis = book.groupby('payment_method')['amount'].agg(['count', 'mean']).round(2)
print(payment_analysis)

In [ ]:
# Visualisation simple
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
book.sample['product_category'].value_counts().plot(kind='bar')
plt.title('Distribution des Catégories de Produits')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
book.sample['amount'].hist(bins=30)
plt.title('Distribution des Montants')
plt.xlabel('Montant')
plt.ylabel('Fréquence')

plt.tight_layout()
plt.show()

<a id="pipeline"></a>
## 4. ⚙️ Génération du Pipeline

Maintenant, générons le pipeline à partir de notre exploration.

In [ ]:
# Génération du pipeline
pipeline = book.concatbook()

print("🔧 Pipeline généré !")
print(f"📝 Nombre d'opérations : {len(book.history)}")
print("\n📋 Historique des opérations :")
for i, operation in enumerate(book.history, 1):
    print(f"{i}. {operation['operation']} - {operation['arguments']}")

print("\n⚙️ Pipeline structure :")
print(pipeline)

<a id="elexdas"></a>
## 5. 🐼 Application avec Elexdas (Pandas)

Appliquons maintenant le pipeline sur le dataset complet avec Elexdas.

In [ ]:
# Note: Elexdas n'est pas encore implémenté, voici une simulation
print("🐼 Simulation d'Elexdas...")
print("\n🔄 Application du pipeline sur le dataset complet...")

# Simulation de l'application du pipeline
processed_df = sales_df.copy()

# Application manuelle des opérations du pipeline
print("1. Suppression des montants manquants...")
processed_df = processed_df.dropna(subset=['amount'])
print(f"   ✅ {len(processed_df):,} lignes restantes")

print("2. Remplissage des catégories manquantes...")
processed_df['product_category'] = processed_df['product_category'].fillna('Unknown')
print("   ✅ Catégories remplies")

print("3. Remplissage des notes manquantes...")
median_rating_full = processed_df['customer_rating'].median()
processed_df['customer_rating'] = processed_df['customer_rating'].fillna(median_rating_full)
print(f"   ✅ Notes remplies avec la médiane ({median_rating_full})")

print("4. Conversion des types...")
processed_df = processed_df.astype({
    'amount': 'float64',
    'customer_rating': 'int64',
    'discount_applied': 'bool'
})
print("   ✅ Types convertis")

print(f"\n🎯 Dataset final : {len(processed_df):,} lignes")
print(f"💾 Taille en mémoire : {processed_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
# Comparaison avant/après
print("📊 Comparaison avant/après traitement :")
print("\nAvant :")
print(sales_df.info())
print("\nAprès :")
print(processed_df.info())

<a id="elexpark"></a>
## 6. ⚡ Application avec Elexpark (Spark)

Pour les très gros volumes, nous pourrions utiliser Elexpark avec Spark.

In [ ]:
# Note: Elexpark n'est pas encore implémenté, voici une simulation
print("⚡ Simulation d'Elexpark avec Spark...")
print("\n💡 Pour des datasets de plusieurs millions de lignes, Elexpark permettrait :")
print("   • Distribution automatique des calculs")
print("   • Optimisation des requêtes")
print("   • Gestion de la mémoire pour les gros volumes")
print("   • Traitement parallèle sur plusieurs machines")

# Exemple de code Elexpark (quand il sera implémenté)
spark_code = '''
# Code Elexpark futur
from pyspark.sql import SparkSession
from datalexir.elexpark import Elexpark

# Initialisation Spark
spark = SparkSession.builder.appName("DataLexir").getOrCreate()
spark_df = spark.createDataFrame(sales_df)

# Application du pipeline
elexpark = Elexpark(spark_df)
result_spark = elexpark.pipeline(pipeline)

# Collecte des résultats
final_df = result_spark.toPandas()
'''

print("\n📝 Exemple de code Elexpark :")
print(spark_code)

<a id="conclusion"></a>
## 7. 🎯 Conclusion

Cette démonstration montre le workflow complet de DataLexir :

In [ ]:
print("🎯 Résumé du workflow DataLexir :")
print("\n1. 📊 Dataset original : 100,000 lignes")
print(f"2. 🔍 Exploration sur échantillon : {len(book.sample):,} lignes (5%)")
print(f"3. 🧹 Nettoyage interactif : {len(book.history)} opérations")
print(f"4. ⚙️ Pipeline généré automatiquement")
print(f"5. 🐼 Application avec Pandas : {len(processed_df):,} lignes traitées")
print(f"6. ⚡ Possibilité d'utiliser Spark pour de plus gros volumes")

print("\n✨ Avantages de DataLexir :")
print("   • Exploration rapide sur échantillons")
print("   • Interface interactive dans Jupyter")
print("   • Génération automatique de pipelines")
print("   • Reproductibilité garantie")
print("   • Scalabilité avec Pandas et Spark")
print("   • Traçabilité complète des transformations")

print("\n🚀 DataLexir : De l'exploration à la production en quelques lignes !")

---

## 📚 Ressources supplémentaires

- [Documentation DataLexir](https://github.com/remiv1/data_elexir)
- [Guide de contribution](https://github.com/remiv1/data_elexir/blob/main/CONTRIBUTING.md)
- [Exemples supplémentaires](https://github.com/remiv1/data_elexir/tree/main/notebooks)

**N'hésitez pas à ⭐ le projet sur GitHub si DataLexir vous aide !**